# Week4授業課題
## 信用情報の学習
Kaggleの Home Credit Default Risk コンペティションの学習および推定を行います。

# 【問題1】
## コンペティション内容の確認
コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。

- 何を学習し、何を予測するのか
- どのようなファイルを作りKaggleに提出するか
- 提出されたものはどういった指標値で評価されるのか

## 何を学習し、何を予測するのか
ローンの情報(車を持ってるか、不動産を持ってるか、学歴など)を

クライアントのローンの返済能力を予測する

## どのようなファイルを作りKaggleに提出するか
(TARGETは予測結果(%))

下記のような形式(csvファイル)
```
SK_ID_CURR,TARGET
100001,0.1
100005,0.9
100013,0.2
etc.
```

## 提出されたものはどういった指標値で評価されるのか
ROC曲線を用いて評価される

まずは単純な方法による **ベースラインモデル** を作成します。精度の基準となるモデルです。

精度を高くする必要はありません。エラーなく実行でき、Kaggle側に推定値を提出できることを目指します。

# 【問題2】
## 学習と検証
データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。

検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('application_train.csv')
df_test = pd.read_csv('application_test.csv')
display(df_train)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
display(df_train.columns)

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [4]:
from sklearn.model_selection import train_test_split

df_train_dropna = df_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'TARGET']].dropna(how='any')

X = df_train_dropna[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].values
y = df_train_dropna['TARGET'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(82191, 3) (27398, 3) (82191,) (27398,)


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

print(np.mean(X_train_std[:, 0]), np.var(X_train_std[:, 0]))
print(np.mean(X_train_std[:, 1]), np.var(X_train_std[:, 1]))
print(np.mean(X_train_std[:, 2]), np.var(X_train_std[:, 2]))

-2.3923311747831717e-14 0.9999999999999934
2.3671833555644598e-14 1.0000000000000018
1.1905243344401902e-13 0.9999999999999867


In [6]:
from sklearn.ensemble import RandomForestClassifier
rforest = RandomForestClassifier()
rforest.fit(X_train_std, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [7]:
y_train_pred = rforest.predict_proba(X_train_std)
print(y_train_pred.shape, y_train_pred)
print(y_train.shape, y_train)

(82191, 2) [[0.99 0.01]
 [1.   0.  ]
 [0.32 0.68]
 ...
 [0.95 0.05]
 [0.95 0.05]
 [1.   0.  ]]
(82191,) [0 0 1 ... 0 0 0]


In [8]:
from sklearn.metrics import roc_auc_score
print(f'AUC_train: {roc_auc_score(y_train, y_train_pred[:, 1])}')

AUC_train: 1.0


In [9]:
y_test_pred = rforest.predict_proba(X_test_std)
print(f'AUC_test: {roc_auc_score(y_test, y_test_pred[:, 1])}')

AUC_test: 0.6834289510985117


# 【問題3】
## テストデータに対する推定
テストデータ（`application_test.csv`）に対して推定を行い、Kaggleに提出を行ってください。

正しく提出が行えていれば、精度は低くても構いません。

In [29]:
df_test[['SK_ID_CURR', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].isnull().sum()

SK_ID_CURR          0
EXT_SOURCE_1    20532
EXT_SOURCE_2        8
EXT_SOURCE_3     8668
dtype: int64

In [30]:
# 欠損値の0埋め
df_test_fillna = df_test[['SK_ID_CURR', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].fillna(0)

# SK_ID_CURR
ids = df_test_fillna['SK_ID_CURR'].values
X_test2 = df_test_fillna[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].values

# 標準化
scaler.fit(X_test2)
X_test2_std = scaler.transform(X_test2)

# 予測
y_test2_pred = rforest.predict_proba(X_test2_std)

print(y_test2_pred, y_test2_pred.shape)
print(ids, ids.shape)

submitting = np.concatenate((ids.reshape(-1, 1), y_test2_pred[:, 1].reshape(-1, 1)), axis=1)
print(submitting) 

[[0.91 0.09]
 [1.   0.  ]
 [0.99 0.01]
 ...
 [0.97 0.03]
 [0.99 0.01]
 [0.93 0.07]] (48744, 2)
[100001 100005 100013 ... 456223 456224 456250] (48744,)
[[1.00001e+05 9.00000e-02]
 [1.00005e+05 0.00000e+00]
 [1.00013e+05 1.00000e-02]
 ...
 [4.56223e+05 3.00000e-02]
 [4.56224e+05 1.00000e-02]
 [4.56250e+05 7.00000e-02]]


In [31]:
df_submitting = pd.DataFrame(submitting, columns=['SK_ID_CURR', 'TARGET'])
df_submitting['SK_ID_CURR'] = df_submitting['SK_ID_CURR'].astype(np.int64)
display(df_submitting)
df_submitting.info()

df_submitting.to_csv('test_pred.csv', index=False)

,SK_ID_CURR,TARGET
0,100001,0.09
1,100005,0.00
2,100013,0.01
3,100028,0.02
4,100038,0.22
...,...,...
48739,456221,0.03
48740,456222,0.03
48741,456223,0.03
48742,456224,0.01


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SK_ID_CURR  48744 non-null  int64  
 1   TARGET      48744 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 761.8 KB


ベースラインモデルを元に、入力する特徴量に様々な工夫を行い精度を向上させていきます。

# 【問題4】
## 特徴量エンジニアリング
精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。

- どの特徴量を使うか
- どう前処理をするか

何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。

精度が高かったものに関してはテストデータに対しても推定を行い、Kaggleに提出を行ってください。